In [1]:
from codebase.classes import Data, Particles
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp



## 2 factor Sim AZ

In [2]:
existing_directory = None
task_handle = 'sim_az'
gen_model = 0

if existing_directory is None:
    log_dir = make_folder(task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)




Creating new directory: ./log/20201217_191031_sim_az/


In [3]:
# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 4, 
    size = 100,
    random_seed = 0
    )
    
exp_data.generate()

In [4]:
model_num = 3
## setup particles
param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
particles = Particles(
    name = 'normal',
    model_num = model_num,
    size = 2000,
    param_names = param_names,
    latent_names = latent_names)
particles.set_log_dir(log_dir)
if gen_model:
    particles.compile_prior_model()
    particles.compile_model()
else:
    particles.load_prior_model()
    particles.load_model()

particles.sample_prior_particles(exp_data.get_stan_data()) # sample prior particles
particles.reset_weights() # set weights to 0
log_lklhds = np.empty(exp_data.size)
degeneracy_limit = 0.5
for t in tqdm(range(exp_data.size)):
    particles.get_incremental_weights(
        exp_data.get_stan_data_at_t(t)
        )
    log_lklhds[t] =  particles.get_loglikelihood_estimate()
    particles.update_weights()
    
    if (essl(particles.weights) < degeneracy_limit * particles.size) and (t+1) < exp_data.size:
        particles.resample_particles()
        particles.jitter(exp_data.get_stan_data_upto_t(t+1))
        particles.reset_weights()
    else:
        particles.update_weights()

    save_obj(particles, 'particles', log_dir)
    

100%|██████████| 100/100 [2:35:06<00:00, 93.06s/it]  


In [5]:

save_obj(log_lklhds, 'log_lklhds', log_dir)

print('\n\n')
marg_lklhd = np.exp(logsumexp(log_lklhds))
print('Marginal Likelihood %.5f'%marg_lklhd)

for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(particles.particles[name])
    w = exp_and_normalise(particles.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))





Marginal Likelihood 0.09590


Estimate
[-0.06 -0.08 -0.02 -0.07 -0.08  0.  ]


Estimate
[[0.85 0.33 0.46 0.04 0.07 0.01]
 [0.33 0.91 0.36 0.06 0.07 0.04]
 [0.46 0.36 1.04 0.04 0.09 0.03]
 [0.04 0.06 0.04 0.96 0.41 0.42]
 [0.07 0.07 0.09 0.41 0.93 0.38]
 [0.01 0.04 0.03 0.42 0.38 1.  ]]


In [6]:
marg_lklhd

0.09589759151698887